In [1]:
import mysql.connector
import pandas as pd
mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    passwd = 'jh394828',
    database = 'Turningprocess',
)

mycursor = mydb.cursor()
mycursor.execute('select * from zer3values')
zer3values = mycursor.fetchall()
zer3values = list(zer3values)
data = pd.DataFrame(zer3values,columns=['idZER3val','TimeStamp',\
        'MeanForceX','MeanForceY','MeanForceZ','StdForceX','StdForceY',\
        'StdForceZ','MeanTemp1','MeanTemp2','MeanTemp3','StdTemp1',\
        'StdTemp2','StdTemp3','MeanPower','StdPower','MeanCurrent',\
        'StdCurrent','Ra','Rt','Rz','LaserMean','LaserOnOff',\
        'ExperimentData_idcut'])

mycursor.execute('select * from experimentdata')
experimentdata = mycursor.fetchall()
experimentdata = list(experimentdata)
data_experiment = pd.DataFrame(experimentdata,columns=['idcut','Date','Time','Generation','Offspring','DiameterCNC',\
                    'WorkpieceLength','MeasuredD1','MeasuredD2','MeasuredD3','CuttingInsert','Edge','Gamma',\
                    'Vc','f','ap','ChippedVolume','HS','NS','SK','Aufbau','Ra1','Ra2','Ra3','Rz1','Rz2','Rz3',\
                    'Rt1','Rt2','Rt3','ChipFormation','ChipFormationMR','HRC1','HRC2','HRC3','Remarks','RemarksMR',\
                    'Workpiece_idworkpiece','Machine_idmachine','Tool_idtool'])
mycursor.close()
mydb.close()

In [23]:
import numpy as np
timestamp = []
toolwear=[]
lossdata=[]
for i in data_experiment.idcut:
    if not data[data.ExperimentData_idcut==i].empty:
        #添加时间
        temp_dataExp = data_experiment.iloc[i-1]
        if (temp_dataExp.Vc!=0) and (temp_dataExp.f!=0):
            temp_dataZer = data[data.ExperimentData_idcut==i].iloc[-1]    
            toolwear.append((temp_dataExp.HS+temp_dataExp.NS+temp_dataExp.SK)/3.)
            if len(timestamp) == 0:
                finaldata = pd.concat([temp_dataExp,temp_dataZer]).to_frame().T
                timestamp.append(float(temp_dataZer['TimeStamp']))
                temp_Vc = float(temp_dataExp['Vc'])
                temp_f = float(temp_dataExp['f'])
                temp_Gamma = float(temp_dataExp['Gamma'])
            elif (temp_Vc == float(temp_dataExp['Vc'])) and (temp_f==float(temp_dataExp['f'])) \
                                                        and (temp_Gamma==float(temp_dataExp['Gamma'])):
                temp_time = timestamp[-1] + float(temp_dataZer['TimeStamp'])
                timestamp.append(temp_time)
            else:
                timestamp.append(float(temp_dataZer['TimeStamp']))
                temp_Vc = float(temp_dataExp['Vc'])
                temp_f = float(temp_dataExp['f'])
                temp_Gamma = float(temp_dataExp['Gamma'])
            #合并   
            if i<=2:
                pass
            else:
                finaldata = finaldata.append(pd.concat([temp_dataExp,temp_dataZer]).to_frame().T,ignore_index=True)
    else:
        lossdata.append(i)
toolwear = np.array(toolwear).reshape(len(toolwear),1)
toolwear = pd.DataFrame(toolwear,columns=['ToolWear'])
timestamp = np.array(timestamp).reshape(len(timestamp),1)
timestamp = pd.DataFrame(timestamp,columns=['CuttingTime'])
finaldata = finaldata.loc[:,['Vc','f','Gamma','MeanForceX','MeanForceY','MeanForceZ','MeanTemp1','MeanTemp2',\
                            'MeanTemp3','MeanCurrent','MeanPower']]
finaldata = pd.concat([finaldata,timestamp],axis=1)
finaldata = pd.concat([finaldata,toolwear],axis=1)
finaldata.index = finaldata.index+1
finaldata.to_csv(r'/Users/jianghua/Desktop/hj_internship/data/ordered_data.csv',index_label='index_label')